# Apache Iceberg Write Audit Publish (or Rollback) Pattern

## Create SparkSession

In [73]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSession-Iceberg") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.0") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.type", "hive") \
    .config("spark.sql.catalog.iceberg.uri", "thrift://hive-metastore:9083") \
    .getOrCreate()

spark

In [382]:
spark.sql("DROP TABLE IF EXISTS iceberg.dunder_mifflin.dim_employee")

spark.sql("CREATE DATABASE IF NOT EXISTS dunder_mifflin")

DataFrame[]

In [383]:
spark.sql(
"""
CREATE TABLE IF NOT EXISTS iceberg.dunder_mifflin.dim_employee (
    employee_id integer,
    first_name string,
    last_name string,
    department string,
    start_date date,
    end_date date
)
USING iceberg
LOCATION 's3a://lasagna-workspace-bronze/dunder_mifflin/dim_employee/'
"""
)

print(
    spark.sql(
        "SHOW CREATE TABLE iceberg.dunder_mifflin.dim_employee"
    ).collect()[0]["createtab_stmt"]
)

spark.sql(
"""
INSERT INTO iceberg.dunder_mifflin.dim_employee VALUES
    (1, 'Michael', 'Scott', 'Management', DATE '2005-03-24', NULL),
    (2, 'Jim', 'Halpert', 'Sales', DATE '2005-03-24', NULL),
    (3, 'Pam', 'Beesly', 'Reception', DATE '2005-03-24', NULL)
"""
)

spark.sql(
"""
SELECT * 
FROM iceberg.dunder_mifflin.dim_employee
"""
).show(truncate=False)

CREATE TABLE iceberg.dunder_mifflin.dim_employee (
  employee_id INT,
  first_name STRING,
  last_name STRING,
  department STRING,
  start_date DATE,
  end_date DATE)
USING iceberg
LOCATION 's3a://lasagna-workspace-bronze/dunder_mifflin/dim_employee'
TBLPROPERTIES (
  'current-snapshot-id' = 'none',
  'format' = 'iceberg/parquet',
  'format-version' = '2',
  'write.parquet.compression-codec' = 'zstd')

+-----------+----------+---------+----------+----------+--------+
|employee_id|first_name|last_name|department|start_date|end_date|
+-----------+----------+---------+----------+----------+--------+
|1          |Michael   |Scott    |Management|2005-03-24|NULL    |
|2          |Jim       |Halpert  |Sales     |2005-03-24|NULL    |
|3          |Pam       |Beesly   |Reception |2005-03-24|NULL    |
+-----------+----------+---------+----------+----------+--------+



In [384]:
spark.sql(
"""
ALTER TABLE iceberg.dunder_mifflin.dim_employee
    SET TBLPROPERTIES ('write.wap.enabled'='true')
"""
)

spark.sql(
"""
SHOW TBLPROPERTIES iceberg.dunder_mifflin.dim_employee
"""
).show(truncate=False)

+-------------------------------+-------------------+
|key                            |value              |
+-------------------------------+-------------------+
|current-snapshot-id            |6771953413010099889|
|format                         |iceberg/parquet    |
|format-version                 |2                  |
|write.parquet.compression-codec|zstd               |
|write.wap.enabled              |true               |
+-------------------------------+-------------------+



In [385]:
spark.sql(
""" 
ALTER TABLE iceberg.dunder_mifflin.dim_employee
CREATE BRANCH IF NOT EXISTS `audit` RETAIN 7 DAYS
"""
)

spark.sql(
""" 
ALTER TABLE iceberg.dunder_mifflin.dim_employee
CREATE BRANCH IF NOT EXISTS `quarantine` RETAIN 7 DAYS
"""
)

spark.sql(
""" 
SELECT * 
FROM iceberg.dunder_mifflin.dim_employee.refs
"""
).show(truncate=False)

+----------+------+-------------------+-----------------------+---------------------+----------------------+
|name      |type  |snapshot_id        |max_reference_age_in_ms|min_snapshots_to_keep|max_snapshot_age_in_ms|
+----------+------+-------------------+-----------------------+---------------------+----------------------+
|main      |BRANCH|6771953413010099889|NULL                   |NULL                 |NULL                  |
|audit     |BRANCH|6771953413010099889|604800000              |NULL                 |NULL                  |
|quarantine|BRANCH|6771953413010099889|604800000              |NULL                 |NULL                  |
+----------+------+-------------------+-----------------------+---------------------+----------------------+



In [386]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from datetime import date

new_data = [
    (1, "Michael", "Scott", "Management", date(2005, 3, 24), None), # Não muda nada
    (2, "Jim", "Halpert", "Management", date(2005, 3, 24), None), # Mudou de área
    (3, "Pam", "Beesly Halpert", "Reception", date(2005, 3, 24), None), # Mudou de nome
    (4, "Dwight", "Schrute", "Sales", date(2005, 4, 26), None) # Entrou na empresA
]

schema = StructType([
    StructField("employee_id", IntegerType(), nullable=False),
    StructField("first_name", StringType(), nullable=False),
    StructField("last_name", StringType(), nullable=False),
    StructField("department", StringType(), nullable=False),
    StructField("start_date", DateType(), nullable=False),
    StructField("end_date", DateType(), nullable=True)
])

new_df = spark.createDataFrame(new_data, schema)

new_df.show(truncate=False)

+-----------+----------+--------------+----------+----------+--------+
|employee_id|first_name|last_name     |department|start_date|end_date|
+-----------+----------+--------------+----------+----------+--------+
|1          |Michael   |Scott         |Management|2005-03-24|NULL    |
|2          |Jim       |Halpert       |Management|2005-03-24|NULL    |
|3          |Pam       |Beesly Halpert|Reception |2005-03-24|NULL    |
|4          |Dwight    |Schrute       |Sales     |2005-04-26|NULL    |
+-----------+----------+--------------+----------+----------+--------+



In [387]:
spark.sql(
"""
SELECT * 
FROM iceberg.dunder_mifflin.dim_employee.snapshots
"""
).show(truncate=False)

last_snapshot = spark.sql(
"""
SELECT * 
FROM iceberg.dunder_mifflin.dim_employee.snapshots
"""
).collect()[-1]['snapshot_id']

print(f"Last snapshot: {last_snapshot}")

+-----------------------+-------------------+---------+---------+----------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id|operation|manifest_list                                                                                                                           |summary                                                                                                                          

In [388]:
spark.conf.set("spark.wap.branch", "audit")

new_df.createOrReplaceTempView("new_dataframe")

spark.sql(
"""
MERGE INTO iceberg.dunder_mifflin.dim_employee AS OLD
USING new_dataframe AS NEW
ON OLD.employee_id = NEW.employee_id AND OLD.end_date IS NULL
WHEN MATCHED AND (
    NEW.first_name IS NOT NULL AND (
        OLD.first_name != NEW.first_name 
        OR OLD.last_name != NEW.last_name 
        OR OLD.department != NEW.department
        )
    ) THEN 
    UPDATE SET end_date = CURRENT_DATE()
"""
)

spark.sql(
"""
MERGE INTO iceberg.dunder_mifflin.dim_employee AS OLD
USING new_dataframe AS NEW
ON OLD.employee_id = NEW.employee_id AND OLD.end_date IS NULL
WHEN NOT MATCHED THEN
    INSERT (employee_id, first_name, last_name, department, start_date, end_date)
    VALUES (NEW.employee_id, NEW.first_name, NEW.last_name, NEW.department, CURRENT_DATE(), NULL)

"""
)

DataFrame[]

In [389]:
# spark.read.option("BRANCH", "audit").format("iceberg").load("iceberg.dunder_mifflin.dim_employee").show()

print("branch :: [audit]")
spark.sql(
"""
SELECT *
FROM iceberg.dunder_mifflin.dim_employee
FOR VERSION AS OF 'audit'
"""
).show()


# spark.read.option("BRANCH", "audit").format("iceberg").load("iceberg.dunder_mifflin.dim_employee").show()

print("branch :: [main]")
spark.sql(
"""
SELECT *
FROM iceberg.dunder_mifflin.dim_employee
FOR VERSION AS OF 'main'
"""
).show()

# spark.read.option("BRANCH", "quarantine").format("iceberg").load("iceberg.dunder_mifflin.dim_employee").show()

print("branch :: [quarantine]")
spark.sql(
"""
SELECT *
FROM iceberg.dunder_mifflin.dim_employee
FOR VERSION AS OF 'quarantine'
"""
).show()

branch :: [audit]
+-----------+----------+--------------+----------+----------+----------+
|employee_id|first_name|     last_name|department|start_date|  end_date|
+-----------+----------+--------------+----------+----------+----------+
|          1|   Michael|         Scott|Management|2005-03-24|      NULL|
|          3|       Pam|        Beesly| Reception|2005-03-24|2024-12-18|
|          2|       Jim|       Halpert|     Sales|2005-03-24|2024-12-18|
|          2|       Jim|       Halpert|Management|2024-12-18|      NULL|
|          3|       Pam|Beesly Halpert| Reception|2024-12-18|      NULL|
|          4|    Dwight|       Schrute|     Sales|2024-12-18|      NULL|
+-----------+----------+--------------+----------+----------+----------+

branch :: [main]
+-----------+----------+---------+----------+----------+--------+
|employee_id|first_name|last_name|department|start_date|end_date|
+-----------+----------+---------+----------+----------+--------+
|          1|   Michael|    Scott|Ma

In [374]:
is_good_quality = True

if is_good_quality:
    branch_to_merge = "main"
elif not is_good_quality:
    branch_to_merge = "quarantine"
    

spark.sql(f"""
CALL iceberg.system.fast_forward(
    'iceberg.dunder_mifflin.dim_employee',
    '{branch_to_merge}',
    'audit'
)
""").show()


print("branch :: [audit]")
spark.sql(
"""
SELECT *
FROM iceberg.dunder_mifflin.dim_employee
FOR VERSION AS OF 'audit'
"""
).show()


print("branch :: [main]")
spark.sql(
"""
SELECT *
FROM iceberg.dunder_mifflin.dim_employee
FOR VERSION AS OF 'main'
"""
).show()


print("branch :: [quarantine]")
spark.sql(
"""
SELECT *
FROM iceberg.dunder_mifflin.dim_employee
FOR VERSION AS OF 'quarantine'
"""
).show()


+--------------+-------------------+-------------------+
|branch_updated|       previous_ref|        updated_ref|
+--------------+-------------------+-------------------+
|          main|6102912137278375588|4248769185782717426|
+--------------+-------------------+-------------------+

branch :: [audit]
+-----------+----------+--------------+----------+----------+----------+
|employee_id|first_name|     last_name|department|start_date|  end_date|
+-----------+----------+--------------+----------+----------+----------+
|          2|       Jim|       Halpert|Management|2024-12-18|      NULL|
|          3|       Pam|Beesly Halpert| Reception|2024-12-18|      NULL|
|          4|    Dwight|       Schrute|     Sales|2024-12-18|      NULL|
|          1|   Michael|         Scott|Management|2005-03-24|      NULL|
|          3|       Pam|        Beesly| Reception|2005-03-24|2024-12-18|
|          2|       Jim|       Halpert|     Sales|2005-03-24|2024-12-18|
+-----------+----------+--------------+

In [375]:
spark.sql(
""" 
SELECT * 
FROM iceberg.dunder_mifflin.dim_employee.snapshots
"""
).show(truncate=False)

+-----------------------+-------------------+-------------------+---------+----------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id          |operation|manifest_list                                                                                                                           |summary                           

In [376]:
spark.sql(
f"""
CALL iceberg.system.rollback_to_snapshot(
    'iceberg.dunder_mifflin.dim_employee',
    {last_snapshot}
)
"""
).show(truncate=False)


is_good_quality = False

if is_good_quality:
    branch_to_merge = "main"
elif not is_good_quality:
    branch_to_merge = "quarantine"
    

spark.sql(f"""
CALL iceberg.system.fast_forward(
    'iceberg.dunder_mifflin.dim_employee',
    '{branch_to_merge}',
    'audit'
)
""").show()


print("branch :: [audit]")
spark.sql(
"""
SELECT *
FROM iceberg.dunder_mifflin.dim_employee
FOR VERSION AS OF 'audit'
"""
).show()


print("branch :: [main]")
spark.sql(
"""
SELECT *
FROM iceberg.dunder_mifflin.dim_employee
FOR VERSION AS OF 'main'
"""
).show()


print("branch :: [quarantine]")
spark.sql(
"""
SELECT *
FROM iceberg.dunder_mifflin.dim_employee
FOR VERSION AS OF 'quarantine'
"""
).show()

+--------------------+-------------------+
|previous_snapshot_id|current_snapshot_id|
+--------------------+-------------------+
|4248769185782717426 |6102912137278375588|
+--------------------+-------------------+

+--------------+-------------------+-------------------+
|branch_updated|       previous_ref|        updated_ref|
+--------------+-------------------+-------------------+
|    quarantine|6102912137278375588|4248769185782717426|
+--------------+-------------------+-------------------+

branch :: [audit]
+-----------+----------+--------------+----------+----------+----------+
|employee_id|first_name|     last_name|department|start_date|  end_date|
+-----------+----------+--------------+----------+----------+----------+
|          1|   Michael|         Scott|Management|2005-03-24|      NULL|
|          3|       Pam|        Beesly| Reception|2005-03-24|2024-12-18|
|          2|       Jim|       Halpert|     Sales|2005-03-24|2024-12-18|
|          2|       Jim|       Halpert|Man

In [380]:
spark.sql(
""" 
SELECT * 
FROM iceberg.dunder_mifflin.dim_employee.history
"""
).show(truncate=False)

+-----------------------+-------------------+-------------------+-------------------+
|made_current_at        |snapshot_id        |parent_id          |is_current_ancestor|
+-----------------------+-------------------+-------------------+-------------------+
|2024-12-18 21:25:22.724|6102912137278375588|NULL               |true               |
|2024-12-18 21:26:20.958|4248769185782717426|6318101716745908871|false              |
|2024-12-18 21:27:48.943|6102912137278375588|NULL               |true               |
+-----------------------+-------------------+-------------------+-------------------+

